In [ ]:
import os
import yaml
from pathlib import Path

In [ ]:
def list_all_file_in_folder(folder: Path = '.') -> list[str]:
	file_list = []
	for item in os.listdir(folder):
		item_path = os.path.join(folder, item)
		if os.path.isfile(item_path):
			file_list.append(item_path)
		elif os.path.isdir(item_path):
			file_list.extend(list_all_file_in_folder(item_path))
	file_list = [i.replace('\\', '/') for i in file_list]
	return file_list

def load_yaml(file_path):
	with open(file_path, 'r') as file:
		data = yaml.safe_load(file)
	return data

def save_yaml(data, file_path):
	with open(file_path, 'w') as file:
		yaml.dump(data, file, default_flow_style=False, sort_keys=False)
def load_text_file(file_path):
	with open(file_path, 'r') as file:
		text = file.read()
	return text

def save_text_file(text, file_path):
	with open(file_path, 'w') as file:
		file.write(text)

In [ ]:
COMMON_REMOVE_FILENAME = {
	'.helmignore' 	: 1,
  # '_helpers.tpl'	: 1,
 	'NOTES.txt' 		: 1,
	'README.md' 		: 1,
}

BITNAMI_LICENSE = '''{{- /*
Copyright VMware, Inc.
SPDX-License-Identifier: APACHE-2.0
*/}}
'''

CHART_REMOVE_FIELD = [ 'annotations', 'home', 'icon', 'keywords', 'maintainers']

COMMENT_TEMPLATE = [
	'                    # ',
	'                  # ',
	'                # ',
	'              # ',
	'            # ',
	'          # ',
	'        # ',
	'      # ',
	'    # ',
	'  # ',
	'                    ## ',
	'                  ## ',
	'                ## ',
	'              ## ',
	'            ## ',
	'          ## ',
	'        ## ',
	'      ## ',
	'    ## ',
	'  ## ',
	'## ',
	'        ##',
	'      ##',
	'    ##',
	'  ##',
	'##',
	'# ',
]

def clean_chart_yaml(file_path):
	yaml_data = load_yaml(file_path)
	try:
		for fieldname in CHART_REMOVE_FIELD:
			del yaml_data[fieldname]
	except:
		pass
	save_yaml(yaml_data, file_path)
	return

def clean_bitnami_license(file_path):
  file_content = load_text_file(file_path)
  file_content = file_content.replace(BITNAMI_LICENSE, '')
  save_text_file(file_content, file_path)
  return

def clean_comment(file_path):
  file_content = load_text_file(file_path)
  lines = file_content.split('\n')
  new_lines = []
  for line in lines:
    new_line = line
    for comment_template in COMMENT_TEMPLATE:
      if line.startswith(comment_template):
        new_line = ''
        break
    if new_line != '':
      new_lines.append(new_line)
  file_content = '\n'.join(new_lines)
  save_text_file(file_content, file_path)

def commom_process(chart: Path):
  all_files = list_all_file_in_folder(chart)
  for file in all_files:
    filename = file.split('/')[-1]
    
    if filename == 'Chart.yaml':
      clean_chart_yaml(file)
    
    clean_bitnami_license(file)
    clean_comment(file)
    
    if filename in COMMON_REMOVE_FILENAME:
      os.remove(file)
    
commom_process('argo-cd')